<a
href="https://colab.research.google.com/github/wingated/cs474_labs_f2019/blob/master/DL_Lab6.ipynb"
  target="_parent">
  <img
    src="https://colab.research.google.com/assets/colab-badge.svg"
    alt="Open In Colab"/>
</a>

# Lab 6: Sequence-to-sequence models

### Description:
For this lab, you will code up the [char-rnn model of Karpathy](http://karpathy.github.io/2015/05/21/rnn-effectiveness/). This is a recurrent neural network that is trained probabilistically on sequences of characters, and that can then be used to sample new sequences that are like the original.

This lab will help you develop several new skills, as well as understand some best practices needed for building large models. In addition, we'll be able to create networks that generate neat text!

### Deliverable:
- Fill in the code for the RNN (using PyTorch's built-in GRU).
- Fill in the training loop
- Fill in the evaluation loop. In this loop, rather than using a validation set, you will sample text from the RNN.
- Implement your own GRU cell.
- Train your RNN on a new domain of text (Star Wars, political speeches, etc. - have fun!)

### Grading Standards:
- 20% Implementation the RNN
- 20% Implementation training loop
- 20% Implementation of evaluation loop
- 20% Implementation of your own GRU cell
- 20% Training of your RNN on a domain of your choice

### Tips:
- Read through all the helper functions, run them, and make sure you understand what they are doing
- At each stage, ask yourself: What should the dimensions of this tensor be? Should its data type be float or int? (int is called `long` in PyTorch)
- Don't apply a softmax inside the RNN if you are using an nn.CrossEntropyLoss (this module already applies a softmax to its input).

### Example Output:
An example of my final samples are shown below (more detail in the
final section of this writeup), after 150 passes through the data.
Please generate about 15 samples for each dataset.

<code>
And ifte thin forgision forward thene over up to a fear not your
And freitions, which is great God. Behold these are the loss sub
And ache with the Lord hath bloes, which was done to the holy Gr
And appeicis arm vinimonahites strong in name, to doth piseling
And miniquithers these words, he commanded order not; neither sa
And min for many would happine even to the earth, to said unto m
And mie first be traditions? Behold, you, because it was a sound
And from tike ended the Lamanites had administered, and I say bi
</code>


---

## Part 0: Readings, data loading, and high level training

---

There is a tutorial here that will help build out scaffolding code, and get an understanding of using sequences in pytorch.

* Read the following

> * [Pytorch sequence-to-sequence tutorial](https://pytorch.org/tutorials/intermediate/seq2seq_translation_tutorial.html) (Take note that you will not be implementing the encoder part of this tutorial.)
* [Understanding LSTM Networks](http://colah.github.io/posts/2015-08-Understanding-LSTMs/)






In [1]:
! wget -O ./text_files.tar.gz 'https://piazza.com/redirect/s3?bucket=uploads&prefix=attach%2Fjlifkda6h0x5bk%2Fhzosotq4zil49m%2Fjn13x09arfeb%2Ftext_files.tar.gz'
! tar -xzf text_files.tar.gz
! pip install unidecode
! pip install torch

import unidecode
import string
import random
import re

import pdb

all_characters = string.printable
n_characters = len(all_characters)
file = unidecode.unidecode(open('./text_files/lotr.txt').read())
file_len = len(file)
print('file_len =', file_len)

--2023-10-13 20:57:29--  https://piazza.com/redirect/s3?bucket=uploads&prefix=attach%2Fjlifkda6h0x5bk%2Fhzosotq4zil49m%2Fjn13x09arfeb%2Ftext_files.tar.gz
Resolving piazza.com (piazza.com)... 54.83.207.146, 3.228.96.161, 3.222.220.252, ...
Connecting to piazza.com (piazza.com)|54.83.207.146|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-uploads.piazza.com/attach/jlifkda6h0x5bk/hzosotq4zil49m/jn13x09arfeb/text_files.tar.gz [following]
--2023-10-13 20:57:29--  https://cdn-uploads.piazza.com/attach/jlifkda6h0x5bk/hzosotq4zil49m/jn13x09arfeb/text_files.tar.gz
Resolving cdn-uploads.piazza.com (cdn-uploads.piazza.com)... 18.239.225.73, 18.239.225.125, 18.239.225.104, ...
Connecting to cdn-uploads.piazza.com (cdn-uploads.piazza.com)|18.239.225.73|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1533290 (1.5M) [application/x-gzip]
Saving to: ‘./text_files.tar.gz’

./text_files.tar.gz 100%[===================>]   1.46M  --.-KB/

In [2]:
def random_chunk(file, file_len, chunk_len):
  start_index = random.randint(0, file_len - chunk_len)
  end_index = start_index + chunk_len + 1
  return file[start_index:end_index]

In [3]:
import torch
# Turn string into list of longs
def char_tensor(string):
  tensor = torch.zeros(len(string)).long()
  for c in range(len(string)):
      tensor[c] = all_characters.index(string[c])
  return tensor

print(char_tensor('abcDEF'))

tensor([10, 11, 12, 39, 40, 41])


---

## Part 4: Creating your own GRU cell

**(Come back to this later - its defined here so that the GRU will be defined before it is used)**

---

The cell that you used in Part 1 was a pre-defined Pytorch layer. Now, write your own GRU class using the same parameters as the built-in Pytorch class does.

Please try not to look at the GRU cell definition. The answer is right there in the code, and in theory, you could just cut-and-paste it. This bit is on your honor!

**TODO:**

**DONE:**
* Create a custom GRU cell



In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class GRU(nn.Module):
  def __init__(self, input_size, hidden_size, num_layers):
    super(GRU, self).__init__()
    self.input_size = input_size
    self.hidden_size = hidden_size
    self.num_layers = num_layers

    self.W_ir = nn.Linear(input_size, hidden_size)
    self.W_hr = nn.Linear(hidden_size, hidden_size)
    self.W_iz = nn.Linear(input_size, hidden_size)
    self.W_hz = nn.Linear(hidden_size, hidden_size)
    self.W_in = nn.Linear(input_size, hidden_size)
    self.W_hn = nn.Linear(hidden_size, hidden_size)

  def forward(self, inputs, hidden):
    # Each layer does the following:
    # r_t = sigmoid(W_ir*x_t + b_ir + W_hr*h_(t-1) + b_hr)
    # z_t = sigmoid(W_iz*x_t + b_iz + W_hz*h_(t-1) + b_hz)
    # n_t = tanh(W_in*x_t + b_in + r_t**(W_hn*h_(t-1) + b_hn))
    # h_(t) = (1 - z_t)**n_t + z_t**h_(t-1)
    # Where ** is hadamard product (not matrix multiplication, but elementwise multiplication)

    x_t = inputs
    h_t_1 = hidden

    r_t = torch.sigmoid(self.W_ir(x_t) + self.W_hr(h_t_1))
    z_t = torch.sigmoid(self.W_iz(x_t) + self.W_hz(h_t_1))
    n_t = torch.tanh(self.W_in(x_t) + r_t*self.W_hn(h_t_1))
    h_t = (1 - z_t)*n_t + z_t*h_t_1

    outputs = h_t
    hiddens = h_t

    return outputs, hiddens



---

##  Part 1: Building a sequence to sequence model

---

Great! We have the data in a useable form. We can switch out which text file we are reading from, and trying to simulate.

We now want to build out an RNN model, in this section, we will use all built in Pytorch pieces when building our RNN class.


**TODO:**

**DONE:**
* Create an RNN class that extends from nn.Module.



In [5]:
class RNN(nn.Module):
  def __init__(self, input_size, hidden_size, output_size, n_layers=1):
    super(RNN, self).__init__()
    self.input_size = input_size
    self.hidden_size = hidden_size
    self.output_size = output_size
    self.n_layers = n_layers

    # more stuff here...
    self.encoder = nn.Embedding(input_size, hidden_size)
    self.gru1 = GRU(hidden_size, hidden_size, n_layers)
    self.gru2 = GRU(hidden_size, hidden_size, n_layers)
    self.gru3 = GRU(hidden_size, hidden_size, n_layers)
    self.decoder = nn.Linear(hidden_size, output_size)

  def forward(self, input_char, hidden):
    # by reviewing the documentation, construct a forward function that properly uses the output of the GRU

    input_char = self.encoder(input_char.view(1, -1))
    output1, hidden1 = self.gru1(input_char.view(1, 1, -1), hidden[0])
    output2, hidden2 = self.gru2(output1, hidden[1])
    output3, hidden3 = self.gru3(output2, hidden[2])
    output = self.decoder(output3.view(1, -1))
    hidden = torch.cat([hidden1, hidden2, hidden3], dim=0)
    return output, hidden

  def init_hidden(self):
    return torch.zeros(self.n_layers, 1, self.hidden_size)

In [6]:
def random_training_set(file, file_len, chunk_len):
  chunk = random_chunk(file, file_len, chunk_len)
  inp = char_tensor(chunk[:-1])
  target = char_tensor(chunk[1:])
  return inp, target

---

## Part 2: Sample text and Training information

---

We now want to be able to train our network, and sample text after training.

This function outlines how training a sequence style network goes.

**TODO:**

**DONE:**
* Fill in the pieces.




In [7]:
# NOTE: decoder_optimizer, decoder, and criterion will be defined below as global variables
def train(inp, target, decoder, decoder_optimizer):
  # initialize hidden layers, set up gradient and loss

  decoder_optimizer.zero_grad()
  hidden = decoder.init_hidden()
  loss = 0

  for c in range(inp.size(0)):
    output, hidden = decoder(inp[c], hidden)
    loss += criterion(output, target[c].view(-1))

  loss.backward()
  decoder_optimizer.step()

  return loss.item() / inp.size(0)

---

## Part 3: Sample text and Training information

---

You can at this time, if you choose, also write out your train loop boilerplate that samples random sequences and trains your RNN. This will be helpful to have working before writing your own GRU class.

If you are finished training, or during training, and you want to sample from the network you may consider using the following function. If your RNN model is instantiated as `decoder`then this will probabilistically sample a sequence of length `predict_len`

**TODO:**

**DONE:**
* Fill out the evaluate function to generate text frome a primed string



In [8]:
def sample_outputs(output, temperature):
    """Takes in a vector of unnormalized probability weights and samples a character from the distribution"""
    return torch.multinomial(torch.exp(output / temperature), 1)

def evaluate(decoder, prime_str='A', predict_len=100, temperature=0.8):
  # initialize hidden state, initialize other useful variables

  hidden = decoder.init_hidden()
  prime_input = char_tensor(prime_str)
  predicted = prime_str

  for p in range(len(prime_str) - 1):
    _, hidden = decoder(prime_input[p], hidden)
  inp = prime_input[-1]

  for p in range(predict_len):
    output, hidden = decoder(inp, hidden)

    output_dist = output.data.view(-1).div(temperature).exp()
    top_i = torch.multinomial(output_dist, 1)[0]

    predicted_char = all_characters[top_i]
    predicted += predicted_char
    inp = char_tensor(predicted_char)

  return predicted


---

## Part 4: (Create a GRU cell, requirements above)

---



---

## Part 5: Run it and generate some text!

---


**TODO:**

**DONE:**
* Create some cool output




Assuming everything has gone well, you should be able to run the main function in the scaffold code, using either your custom GRU cell or the built in layer, and see output something like this. I trained on the “lotr.txt” dataset, using chunk_length=200, hidden_size=100 for 2000 epochs. These are the results, along with the prime string:

---

 G:

 Gandalf was decrond.
'All have lord you. Forward the road at least walk this is stuff, and
went to the long grey housel-winding and kindled side was a sleep pleasuring, I do long
row hrough. In  

 lo:

 lost death it.
'The last of the gatherings and take you,' said Aragorn, shining out of the Gate.
'Yes, as you there were remembaused to seen their pass, when? What
said here, such seven an the sear

 lo:

 low, and frod to keepn
Came of their most. But here priced doubtless to an Sam up is
masters; he left hor as they are looked. And he could now the long to stout in the right fro horseless of
the like

 I:

 I had been the
in his eyes with the perushed to lest, if then only the ring and the legended
of the less of the long they which as the
enders of Orcovered and smood, and the p

 I:

 I they were not the lord of the hoomes.
Home already well from the Elves. And he sat strength, and we
housed out of the good of the days to the mountains from his perith.

'Yess! Where though as if  

 Th:

 There yarden
you would guard the hoor might. Far and then may was
croties, too began to see the drumbred many line
and was then hoard walk and they heart, and the chair of the
Ents of way, might was

 G:

 Gandalf
been lat of less the round of the stump; both and seemed to the trees and perished they
lay are speered the less; and the wind the steep and have to she
precious. There was in the oonly went

 wh:

 which went out of the door.
Hull the King and of the The days of his brodo
stumbler of the windard was a thing there, then it been shining langing
to him poor land. They hands; though they seemed ou

 ra:

 rather,' have all the least deather
down of the truven beginning to the house of sunk.
'Nark shorts of the Eyes of the Gate your great nothing as Eret.
'I wander trust horn, and there were not, it  

 I:

 I can have no mind
together! Where don't may had one may little blung
terrible to tales. And turn and Gandalf shall be not to as only the Cattring
not stopped great the out them forms. On they she lo

---


In [9]:
import time
n_epochs = 2000
print_every = 200
plot_every = 10
hidden_size = 200
n_layers = 3
lr = 0.001

decoder = RNN(n_characters, hidden_size, n_characters, n_layers)
decoder_optimizer = torch.optim.Adam(decoder.parameters(), lr=lr)
criterion = nn.CrossEntropyLoss()

start = time.time()
all_losses = []
loss_avg = 0

In [10]:
for epoch in range(1, n_epochs + 1):
  loss_ = train(*random_training_set(file, file_len, 200), decoder, decoder_optimizer)
  loss_avg += loss_

  if epoch % print_every == 0:
      print('[%s (%d %d%%) %.4f]' % (time.time() - start, epoch, epoch / n_epochs * 100, loss_))
      print(evaluate(decoder, 'Wh', 100), '\n')

  if epoch % plot_every == 0:
      all_losses.append(loss_avg / plot_every)
      loss_avg = 0

[145.96999192237854 (200 10%) 2.5403]
Whe sacokan'traghrat thain, noln ogow cesed 
ussant hant eafd sandes, the yeurod 
of hratralt lith, Wi 

[286.4713053703308 (400 20%) 2.0941]
Whe, had bey of the mian fordon. Thead in Sajy the anland of the zeilking the lordas. 

And rill hing  

[428.1036605834961 (600 30%) 1.7597]
Whe faes it saaded are had and not great can the daurilen, the no dowred and where on 
gansagh, and th 

[566.169410943985 (800 40%) 1.6900]
Why the shazed, the was fear no lards the math saw have nown of yougSended and ergettoo lept. And the  

[706.0369701385498 (1000 50%) 1.8370]
Where came begrient their spaging in the windered; 'he know foor to falled he dlive and gliel, and see 

[839.5608501434326 (1200 60%) 1.5364]
Whing, and the there the First upon flect 
have that we clanger north, Mrnand spys curl. The foor see  

[976.3331689834595 (1400 70%) 1.5845]
Wher deep even is that swords as beget come.' 




How of this beling his sking and we have to sparlin 

[11

In [13]:
for i in range(10):
  start_strings = [" Th", " wh", " he", " I ", " ca", " G", " lo", " ra"]
  start = random.randint(0,len(start_strings)-1)
  print(start_strings[start])
  # all_characters.index(string[c])
  print(evaluate(decoder, start_strings[start], 200), '\n')

 I 
 I corn. Will reaves and face there we fained they swond 
been far east caves once. 'Go speep he are 
you Baketing by stone looked they seemed of we 
should to stoes and heavil they came I had greeped in 

 Th
 They's shadow had been the foof his prash. 

'They have they will sift him, as I wall freet of may, not that Gimle. 'Where in the black before him, doubt.' 

'Let the glowing once as a crass that Ume. ' 

 he
 hear and sut wholled, simber mares of their hid a 
narrow to staces the wince, and it sacken as if the Sun stream was no could 
in mes, his crowing green were before white be a refuent the Nund. 

'I hi 

 he
 he stringh their into stare found like 
in the but the stood and they is we shadualing that he said. 'I middle of help and fought houses we hill the glame for even a swarves. he 
said and stroming but t 

 I 
 I internof the place up the 
plash to chancts. I 
well, and disho his side deeping them and sut in the 
gloomes to the scorned and a strider to to the poin

---

## Part 6: Generate output on a different dataset

---

**TODO:**

**DONE:**

* Choose a textual dataset. Here are some [text datasets](https://www.kaggle.com/datasets?tags=14104-text+data%2C13205-text+mining) from Kaggle

* Generate some decent looking results



In [14]:
import pandas as pd

my_file = pd.read_csv('./text_files/avatar.csv', encoding='latin1')
my_file = my_file.loc[:, 'character_words']

my_file_text = str()
for i in range(len(my_file)):
  datum = my_file[i]

  if datum != 'NA' and type(datum) is not float:
    for c in datum:
      if c in all_characters:
        my_file_text += c

    my_file_text += '\n'

my_file_len = len(my_file_text)

In [15]:
my_decoder = RNN(n_characters, hidden_size, n_characters, n_layers)
my_decoder_optimizer = torch.optim.Adam(my_decoder.parameters(), lr=lr)
criterion = nn.CrossEntropyLoss()

start = time.time()
my_all_losses = []
my_loss_avg = 0

for epoch in range(1, n_epochs + 1):
  loss_ = train(*random_training_set(my_file_text, my_file_len, 200), my_decoder, my_decoder_optimizer)
  my_loss_avg += loss_

  if epoch % print_every == 0:
      print('[%s (%d %d%%) %.4f]' % (time.time() - start, epoch, epoch / n_epochs * 100, loss_))
      print(evaluate(my_decoder, 'Wh', 100), '\n')

  if epoch % plot_every == 0:
      my_all_losses.append(my_loss_avg / plot_every)
      my_loss_avg = 0

[140.4013946056366 (200 10%) 2.1889]
Whe sea lal joll so of Som weathe and you pant of <eart fean't mat hat to thoncest gat if you you your 

[284.7523338794708 (400 20%) 1.9753]
Whis Kister iled who dat this sens mame. stow for conted unsery will a did to besed is in is wething f 

[430.28567838668823 (600 30%) 1.8721]
What's out!
That's flow you don't ening piss she you like my your know this in is so dest in the ened  

[576.2095651626587 (800 40%) 1.5571]
Whrenst!
I'm now think and I can guZse with in as a chandey.
We're mone in of there.  Aang, you're and 

[720.9189915657043 (1000 50%) 1.9484]
Where. The sand! I'm we sto thim. Don't would go?
What do ope the are not some for the wirebender alli 

[861.8719162940979 (1200 60%) 1.8360]
What?
Uh!  Not to get of your will fi all I want you should see Zuko. Where's and not is cretter?
What 

[1004.4797706604004 (1400 70%) 1.8296]
Where washore.
I knew be nothing on you they got in you were firebender! Well, we mean, I want a sing  

[